In [1]:
root_dir = "/Users/denisegosnell/Desktop/Book/dse-6.8.0/graph-data-book/graphdatabook/data/"
source_dir = root_dir + "source_data/"
destination_dir = root_dir + "movies/"

#source data files\n",
ratings_file = source_dir + "ratings.csv"
links_file = source_dir + "links.csv"
tags_file = source_dir + "tags.csv"
genome_scores_file = source_dir + "genome-scores.csv"
genome_tags_file = source_dir + "genome-tags.csv"
movies_file = source_dir + "movies.csv"
actors_file = source_dir + "AllMoviesCastingRaw.csv"
movie_details_file = source_dir + "AllMoviesDetailsCleaned_properCSV.csv"

# destination data files\n",
movie_vertices = destination_dir + "movie_vertices.csv"
genre_vertices = destination_dir + "genre_vertices.csv"
tag_vertices = destination_dir + "tag_vertices.csv"
user_vertices = destination_dir + "user_vertices.csv"
actor_vertices = destination_dir + "actor_vertices.csv"

genre_edges = destination_dir + "contains_genre.csv"
movie_topic_tag_edges = destination_dir + "movie_topic_tag.csv"
tagged_edges = destination_dir + "tagged.csv"
rated_edges = destination_dir + "rated.csv"
acted_edges = destination_dir + "acted.csv"
collaborator_edges = destination_dir + "collaborator.csv"

In [2]:
dupes =[]
last_movie_id = 0
movies = {}
movies_by_tmbd = {}
movies_by_imbd = {}
missing_imdb = 0
missing_tmdb = 0
clashes = 0
with open(links_file, "r") as read_links:
    next(read_links)
    for line in read_links:
        data = line.replace("\n","").split(",")
        movie_id = int(data[0])
        imdb_id = str(data[1])
        tmdb_id = str(data[2])
        movie_data = {}
        if imdb_id == "":
            missing_imdb += 1
        else: 
            movie_data["imdb_id"] = imdb_id
            movies_by_imbd[imdb_id] = movie_id
        if tmdb_id == "":
            missing_tmdb += 1
        else: 
            # check to see if we have already put this one in the lookup
            if movies_by_tmbd.get(tmdb_id):
                clashes += 1
                # remove this link because it is wrong
                movies_by_tmbd.pop(tmdb_id)
                movie_data["tmdb_id"] = ""
            else:
                movies_by_tmbd[tmdb_id] = movie_id
                movie_data["tmdb_id"] = tmdb_id
        movies[movie_id] = movie_data
        if movie_id > last_movie_id:
            last_movie_id = movie_id

In [3]:
print("Total movies: ", len(movies))
print("Total movies w/ imbd ids: ",len(movies_by_imbd))
print("Total movies w/ tmbd ids: ",len(movies_by_tmbd))
print("Largest movie_id from movie lens: ",last_movie_id)
print("Total clashes between tmbd and imbd: ",clashes)
print("Total  movies from movie lens missing imbd id: ",missing_imdb)
print("Total movies from movie lens missing tmbd id: ",missing_tmdb)


Total movies:  27278
Total movies w/ imbd ids:  27278
Total movies w/ tmbd ids:  26992
Largest movie_id from movie lens:  131262
Total clashes between tmbd and imbd:  17
Total  movies from movie lens missing imbd id:  0
Total movies from movie lens missing tmbd id:  252


In [4]:
assert(len(movies) == 27278)
assert(len(movies_by_imbd) == 27278)
assert(len(movies_by_tmbd) == 26992)
assert(missing_imdb == 0)
assert(missing_tmdb == 252)
assert(last_movie_id == 131262)

In [5]:
genres = set([])
movie_genre = []
with open(movies_file, "r") as read_movies:
    next(read_movies)
    for line in read_movies:
        data = line.replace("\n","").split(",")
        title = str(data[1])
        movie_id = int(data[0])
        if movies.get(movie_id):
            movies[movie_id]["movie_title"] = title
        else:
            movies[movie_id] = {"movie_title": title}
        if movie_id > last_movie_id:
            last_movie_id = movie_id
        genre = data[2].split("|")
        for g in genre:
            genres.add(g)
            movie_genre.append([movie_id,g])

In [6]:
from datetime import datetime
def format_date(date):
    """
    input format: dd/mm/YYYY or dd/mm/yy or YYYY-mm-dd,
    output: ISO8601 Standard
    """
    if date:
        if "-" in date:
            date_obj = datetime.strptime(date, '%Y-%m-%d')
        else:
            try:
                date_obj = datetime.strptime(date, '%d/%m/%Y')
            except ValueError:
                try:
                    date_obj = datetime.strptime(date, '%d/%m/%y')
                except ValueError:
                    return ["not_published", "not_published"]
        return [date_obj.isoformat(), date_obj.year]
    else:
        return ["not_published", "not_published"]


In [7]:
print(format_date("2/9/04"))
print(format_date("30/04/1941"))
print(format_date("1896-01-25"))
print(format_date(""))
print(format_date("none"))

['2004-09-02T00:00:00', 2004]
['1941-04-30T00:00:00', 1941]
['1896-01-25T00:00:00', 1896]
['not_published', 'not_published']
['not_published', 'not_published']


In [8]:
def id_assertion(imdb_id, tmbd_id, bad_id_mappings, title):
    """
        we want to confirm:
               A. movies[movie_id] has matching imdb ids
               B. movies[movie_id] has matching tmdb ids
    """
    movie_id = ""
    try:
        ## check mappings for imbd --> tmbd
        if movies_by_imbd.get(imdb_id):
            movie_id = movies_by_imbd.get(imdb_id)
            potential_error = "L12: imdb_id not in movies[movie_id].keys()"
            assert("imdb_id" in movies[movie_id].keys())
            potential_error = "L14: imdb_id_ml != imdb_id"
            assert(str(imdb_id) == str(movies[movie_id]["imdb_id"]))
            if "tmdb_id" in movies[movie_id].keys():
                potential_error = "L17: movies[movie_id][tmdb_id] != tmbd_id"
                assert(str(tmdb_id) == str(movies[movie_id]["tmdb_id"]))
        ## check mappings for tmbd --> imbd
        if movies_by_tmbd.get(tmdb_id):
            movie_id = movies_by_tmbd.get(tmdb_id)
            potential_error = "L36: tmdb_id not in movies[movie_id].keys()"
            assert("tmdb_id" in movies[movie_id].keys())
            potential_error = "L39: tmdb_id_ml != tmdb_id"
            assert(movies[movie_id]["tmdb_id"] == tmdb_id)
            if "imdb_id" in movies[movie_id].keys():
                potential_error = "L45: imdb_id_ml != imdb_id"
                assert(movies[movie_id]["imdb_id"] == imdb_id)                              
    except AssertionError:
        movie_lens_data = {}
        if movie_id:
            movie_lens_data = {"movie_id": movie_id, "data": movies[movie_id]}
        new_dabase = {"tmdb":tmdb_id, "imdb":imdb_id, "title":title}
        bad_id_mappings.append({"error": potential_error, "movie_lens_data": movie_lens_data, "new_dabase": new_dabase})
    return bad_id_mappings

def get_movie_id(imdb_id, last_movie_id, title, tmdb_id, found_in_movielens):
    """
    """
    found = False
    ## check if the movie is in tmbd_id
    if movies_by_tmbd.get(tmdb_id):
        found = True
        movie_id = movies_by_tmbd.get(tmdb_id)

        ## add data about imbd
        if "imdb_id" not in movies[movie_id].keys():
            # update the new imdb_id
            # our tests of this data showed that we *never* clashed here
            movies[movie_id]["imdb_id"] = imdb_id

    ## check if the movie is in imdb_id
    elif movies_by_imbd.get(imdb_id):
        found = True
        movie_id = movies_by_imbd.get(imdb_id)

        ## add data about tmbd
        if "tmdb_id" in movies[movie_id].keys():
            # update the new imdb_id
            ## TODO -- CHECK IF THEY ARE THE SAME
            ## THERE ARE 143 INSTANCES WHEN IMD IDS MATCH, BUT TM ARE DIFFERENT
            ## Data Analysis: Kaggle has better ground truth testing; adopt the Kaggle Dataset values
            if movies[movie_id]["tmdb_id"] != tmdb_id:
                # overwrite the MovieLens Data with the Kaggle Data
                movies[movie_id]["tmdb_id"] = tmdb_id

    else:
        # this is a movie that was not in the previous database
        last_movie_id = last_movie_id + 1
        movie_id = last_movie_id
        if movies.get(movie_id):
            print("line 114 get_movie_id logic error", movie_id, imdb_id)
            raise ValueError
        movies[movie_id] = {"title": title,
                            "imdb_id": imdb_id,
                            "tmdb_id": tmdb_id}
    if found:
        found_in_movielens += 1
    return [movie_id, last_movie_id, found_in_movielens]


In [9]:
## movies vertex data
# movie_id, tmdb_id, imdb_id, movie_title, 
# release_date production_company, overview
# popularity, budget, revenue
no_tmdb_id = 0
no_imdb_id = 0
total_clean_entires = 0
imdb_from_other = 0
bad_lines = []
bad_id_mappings = []
movies_by_year = {}
found_in_movielens = 0
tmdb_dist = {}
with open(movie_details_file, "r", encoding="ISO-8859-1") as read_movies_details:
    next(read_movies_details)
    for line in read_movies_details:
        try:
            line = line.replace("\n", "").replace("amp\;", "")
            data = line.split(",")
            tmdb_id = int(data[0])
            budget = int(data[1])
            genre = str(data[2]).split("|")
            imdb_id = str(data[3].replace("tt",""))
            if not tmdb_id:
                no_tmdb_id += 1
            if not imdb_id:
                no_imdb_id += 1
  
            # skip language, orig title
            overview = str(data[6])
            try:
                popularity = float(data[7])
            except ValueError:
                popularity = ""
            production_company = str(data[8])
            # skip countries
            [release_date, year] = format_date(data[10])
            revenue = int(data[11])
            title = str(data[16])
            # ignore all other items
            
            bad_id_mappings = id_assertion(imdb_id, tmdb_id, bad_id_mappings, title)
            [movie_id, last_movie_id, found_in_movielens] = get_movie_id(imdb_id, last_movie_id, title, tmdb_id, found_in_movielens)

            movies[movie_id]["release_date"] = release_date
            movies[movie_id]["production_company"] = production_company
            movies[movie_id]["overview"] = overview

            movies[movie_id]["popularity"] = popularity
            movies[movie_id]["budget"] = budget
            movies[movie_id]["revenue"] = revenue
            movies_by_year[tmdb_id] = year
            for g in genre:
                genres.add(g)
                movie_genre.append([movie_id,g])
            total_clean_entires += 1
        except Exception as e:
            bad_lines.append(line)
            print("line 58 main", str(e))
            print(line)
            print(data)
            input()
            pass


In [10]:
error_analysis = {}
for item in bad_id_mappings:
    if item["error"] in error_analysis.keys():
        error_analysis[item["error"]] +=1
    else:
        error_analysis[item["error"]] = 1
    

In [11]:
print("Total movies in both databases ", found_in_movielens)
print("Total movies in new database with no tmdb", no_tmdb_id)
print("Total movies in new database with no imdb", no_imdb_id, "\n")

print("Total bad mappings", len(bad_id_mappings))
print("Type of errors", (error_analysis))
print("Total merged movies: ", len(movies))
print("total new entires from Kaggle", total_clean_entires)

Total movies in both databases  26853
Total movies in new database with no tmdb 0
Total movies in new database with no imdb 78480 

Total bad mappings 143
Type of errors {'L17: movies[movie_id][tmdb_id] != tmbd_id': 143}
Total merged movies:  329469
total new entires from Kaggle 329044


In [14]:
bad_id_mappings[0]

{'error': 'L17: movies[movie_id][tmdb_id] != tmbd_id',
 'movie_lens_data': {'movie_id': 34330,
  'data': {'imdb_id': '0368089',
   'tmdb_id': '9775',
   'movie_title': 'November (2004)',
   'release_date': '2004-07-22T00:00:00',
   'production_company': 'IFC Productions',
   'overview': "Sophie Jacobs is going through the most difficult time of her life. Now  she just has to find out if it's real.",
   'popularity': 0.648027,
   'budget': 0,
   'revenue': 0}},
 'new_dabase': {'tmdb': 1424, 'imdb': '0368089', 'title': 'November'}}

In [ ]:
movies_with_no_imdb_id = 0
movies_with_no_tbdb_id = 0

In [ ]:
interesting = []
for key, val in movies.items():
    if "imdb_id" in val.keys():
        if str(val["imdb_id"]) == "":
            movies_with_no_imdb_id += 1
    else:
        movies_with_no_imdb_id += 1
    if "tmdb_id" in val.keys():
        if str(val["tmdb_id"]) == "":
            movies_with_no_tbdb_id += 1
            interesting.append(key)
    else:
        movies_with_no_tbdb_id += 1
        interesting.append(key)

In [ ]:
print("Final movies with no imdb_id", movies_with_no_imdb_id)
print("Final movies with no tmdb_id", movies_with_no_tbdb_id)
print("Final movies with no tmdb_id", len(interesting))

In [ ]:
popularity = 0
for key in interesting:
    if "popularity" in movies[key].keys():
        popularity +=1 
print("Final Movies with no tmdb_id and from Kaggle", popularity)

In [ ]:
bad_id_mappings[4]

In [16]:
bad_id_mappings[0]

{'error': 'L17: movies[movie_id][tmdb_id] != tmbd_id',
 'movie_lens_data': {'movie_id': 34330,
  'data': {'imdb_id': '0368089',
   'tmdb_id': '9775',
   'movie_title': 'November (2004)',
   'release_date': '2004-07-22T00:00:00',
   'production_company': 'IFC Productions',
   'overview': "Sophie Jacobs is going through the most difficult time of her life. Now  she just has to find out if it's real.",
   'popularity': 0.648027,
   'budget': 0,
   'revenue': 0}},
 'new_dabase': {'tmdb': 1424, 'imdb': '0368089', 'title': 'November'}}

In [25]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    
    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return [distances[-1],len(s2)]

In [52]:
title_analysis = []
titles = []
for item in bad_id_mappings:
    try:
        movie_lens_title = (item["movie_lens_data"]["data"]["movie_title"]).replace("\"","")
        kaggle_title = item["new_dabase"]["title"]
        [distance, total] = levenshteinDistance(movie_lens_title,kaggle_title)
        percent = 100*float(distance/total)
        if percent < 20:
            print("ML imdb", item["movie_lens_data"]["data"]["imdb_id"])
            print("Kaggle imdb", item["new_dabase"]["imdb"])
            print("ML tmdb", item["movie_lens_data"]["data"]["tmdb_id"])
            print("Kaggle tmdb", item["new_dabase"]["tmdb"])
            print("ML Title ", movie_lens_title)
            print("Kaggle Title ", kaggle_title)
            input()
        title_analysis.append([distance, total, percent])
        titles.append([movie_lens_title, "|", kaggle_title])
    except KeyError as e:
        print(str(e))
        print(item)
        input()
        

ML imdb 0112389
Kaggle imdb 0112389
ML tmdb 185441
Kaggle tmdb 26672
ML Title  Thief and the Cobbler
Kaggle Title  The Thief and the Cobbler

ML imdb 0088318
Kaggle imdb 0088318
ML tmdb 163549
Kaggle tmdb 42102
ML Title  Sunday in the Country
Kaggle Title  A Sunday in the Country

ML imdb 1625345
Kaggle imdb 1625345
ML tmdb 101517
Kaggle tmdb 57469
ML Title  Patrice O'Neal: Elephant in the Room (2011)
Kaggle Title  Patrice O'Neal: Elephant in the Room

ML imdb 0050379
Kaggle imdb 0050379
ML tmdb 
Kaggle tmdb 80255
ML Title  Farewell to Arms
Kaggle Title  A Farewell to Arms

ML imdb 1736049
Kaggle imdb 1736049
ML tmdb 
Kaggle tmdb 84198
ML Title  Place at the Table
Kaggle Title  A Place at the Table

ML imdb 0098090
Kaggle imdb 0098090
ML tmdb 69234
Kaggle tmdb 86962
ML Title  Phantom of the Opera
Kaggle Title  The Phantom of the Opera

ML imdb 0015213
Kaggle imdb 0015213
ML tmdb 172538
Kaggle tmdb 188344
ML Title  Cigarette Girl of Mosselprom
Kaggle Title  The Cigarette Girl of Mosselp

KeyboardInterrupt: 

In [38]:
titles

[['November (2004)', '|', 'November'],
 ['Musa the Warrior (Musa) (2001)', '|', 'The Warrior'],
 ['Still Life (Sanxia haoren) (2006)', '|', 'Still Life'],
 ['Trap', '|', 'The Trap'],
 ['King Lear (Korol Lir) (1971)', '|', 'King Lear'],
 ['Camille Claudel (1988)', '|', 'Camille Claudel'],
 ['Fuel (2008)', '|', 'Fuel'],
 ['Bubble', '|', 'The Bubble'],
 ['How I Unleashed World War II (Jak rozpetalem II wojne swiatowa) (1970)',
  '|',
  'How I Unleashed World War II Part II: Following The Arms'],
 ['Deal', '|', 'The Deal'],
 ['Blackout (2007)', '|', 'Blackout'],
 ['Escaflowne: The Movie (Escaflowne) (2000)', '|', 'Escaflowne: The Movie'],
 ['Days Of Darkness (2007)', '|', 'Days of Darkness'],
 ['Otakus in Love (2004)', '|', 'Otakus in Love'],
 ["Love's Long Journey (2005)", '|', "Love's Long Journey"],
 ['Taboo (Gohatto) (1999)', '|', 'Taboo'],
 ['Warrior', '|', 'The Warrior'],
 ['Age of Ignorance', '|', 'Days of Darkness'],
 ['Promise', '|', 'La Promesse'],
 ['Broderskab (Brotherhood) (20

In [44]:
with open("/Users/denisegosnell/Desktop/titles.txt", "w") as write_file:
    for title in titles:
        write_file.write(str(title))
        write_file.write("\n")